# Tutorial — Proposições da Câmara: da API ao corpus de PDFs (com **Python**)

Neste tutorial, voltado às pessoas já habituadas à linguagem nos primeiros encontros, vamos trabalhar com dados de proposições de Projeto de Lei na Câmara dos Deputados, criando um corpus a partir dos dados obtidos na API da Câmara, passando pelo download dos arquivos até a organização de um corpus.

> **Resumo do que vamos fazer**
>
> 1. Consultar a API da Câmara para listar os **PLs de 2025**;
> 2. Buscar os metadados de cada proposição e obter os **id** e endereços dos arquivos originais das proposições de inteiro teor;
> 3. **Baixar** os PDFs de inteiro teor (com fallback via `/proposicoes/{id}/arquivos`);
> 4. Construir um **corpus** a partir dos PDFs para análises textuais.

---


## 0) Preparação do ambiente

Se estiver rodando localmente, instale as dependências (apenas na primeira vez):

```bash
pip install requests pandas pdfminer.six PyPDF2 nltk unidecode
```


In [ ]:
# imports principais
import time
from pathlib import Path
from typing import Optional, Dict, Any, List

import requests
import pandas as pd

# NLP util
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode

# leitura de PDFs (opcional; se não tiver instalado, o notebook segue sem quebrar)
try:
    from pdfminer.high_level import extract_text  # para extrair o texto
except Exception:
    extract_text = None

try:
    from PyPDF2 import PdfReader  # para contagem de páginas
except Exception:
    PdfReader = None

# downloads de recursos do NLTK (somente se ainda não tiver)
try:
    _ = stopwords.words('portuguese')
except LookupError:
    nltk.download('stopwords')
    _ = stopwords.words('portuguese')

pd.__version__


---
## 1) Sessão HTTP robusta e listagem de proposições

Vamos usar uma `requests.Session` com **retry** e `User-Agent` explícito. Em seguida, listamos os PLs de 2025.


In [ ]:
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def make_session() -> requests.Session:
    s = requests.Session()
    s.headers.update({
        "Accept": "application/json",
        "User-Agent": "cebrap-lab-ia/1.0 (+https://github.com/cebrap-lab)"
    })
    retries = Retry(
        total=5, backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"]
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://", HTTPAdapter(max_retries=retries))
    return s

SESSION = make_session()
BASE = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"

def listar_proposicoes(sigla_tipo: str = "PL", ano: int = 2025, itens: int = 100) -> pd.DataFrame:
    params = {"siglaTipo": sigla_tipo, "ano": ano, "ordem": "ASC", "ordenarPor": "id", "itens": itens}
    r = SESSION.get(BASE, params=params, timeout=60)
    r.raise_for_status()
    return pd.DataFrame(r.json().get("dados", []))

df_proposicoes_simples = listar_proposicoes("PL", 2025, itens=100)
df_proposicoes_simples.head()


> **Nota sobre paginação:** para mais de 100 registros, itere com o parâmetro `pagina` e concatene os resultados.


---
## 2) Função para buscar o **detalhe** de uma proposição


In [ ]:
def obter_proposicao_detalhe(pid: int) -> Optional[Dict[str, Any]]:
    url = f"{BASE}/{pid}"
    r = SESSION.get(url, timeout=60)
    if not r.ok:
        return None
    d = r.json().get("dados", {}) or {}
    st = d.get("statusProposicao") or {}
    return {
        "id": d.get("id"),
        "siglaTipo": d.get("siglaTipo"),
        "numero": d.get("numero"),
        "ano": d.get("ano"),
        "ementa": d.get("ementa"),
        "dataApresentacao": d.get("dataApresentacao"),
        "urlInteiroTeor": d.get("urlInteiroTeor"),
        "status_apreciacao": st.get("apreciacao"),
    }

detalhes = []
for pid in df_proposicoes_simples["id"].tolist():
    time.sleep(0.1)  # cortesia com a API
    d = obter_proposicao_detalhe(int(pid))
    if d:
        detalhes.append(d)

df_proposicoes = pd.DataFrame(detalhes)
df_proposicoes.head()


---
## 3) Resolvendo o **URL do PDF** com fallback em `/proposicoes/{id}/arquivos`

Muitas vezes `urlInteiroTeor` está vazio no detalhe. Por isso, buscamos em `/arquivos` e escolhemos um link que seja PDF.


In [ ]:
from typing import List

def listar_arquivos(pid: int) -> List[Dict[str, Any]]:
    url = f"{BASE}/{pid}/arquivos"
    r = SESSION.get(url, timeout=60)
    if not r.ok:
        return []
    return r.json().get("dados", []) or []

def escolher_url_pdf(row: Dict[str, Any]) -> Optional[str]:
    # 1) tenta o campo direto do detalhe
    url = row.get("urlInteiroTeor")
    if url and isinstance(url, str) and url.strip():
        return url

    # 2) cai para /arquivos e procura PDF
    arquivos = listar_arquivos(int(row["id"]))
    campos = ("urlDownload", "urlInteiroTeor", "url")
    for arq in arquivos:
        eh_pdf = (
            (arq.get("formato") or "").lower() == "pdf" or
            str(arq.get("nome") or "").lower().endswith(".pdf") or
            "pdf" in str(arq.get("titulo") or "").lower()
        )
        if not eh_pdf:
            continue
        for c in campos:
            link = arq.get(c)
            if link and isinstance(link, str) and link.strip():
                return link

    # 3) último recurso: primeira URL disponível
    for arq in arquivos:
        for c in campos:
            link = arq.get(c)
            if link and isinstance(link, str) and link.strip():
                return link
    return None

df_proposicoes = df_proposicoes.copy()
df_proposicoes["url_pdf"] = df_proposicoes.apply(escolher_url_pdf, axis=1)
df_proposicoes[["id","urlInteiroTeor","url_pdf"]].head(10)


---
## 4) **Baixar** os PDFs (robusto)

Usamos a mesma `Session` com retries e checamos `Content-Type` quando disponível.


In [ ]:
def baixar_pdf(pid: int, url: str, outdir: str = "inteiro_teor") -> bool:
    Path(outdir).mkdir(parents=True, exist_ok=True)
    destino = Path(outdir) / f"{pid}.pdf"
    try:
        with SESSION.get(url, stream=True, timeout=120, headers={"Accept": "*/*"}) as r:
            r.raise_for_status()
            # Se o servidor não enviar content-type de PDF, seguimos mesmo assim
            with open(destino, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        # tamanho mínimo pra considerar válido
        return destino.exists() and destino.stat().st_size > 1024
    except Exception:
        return False

outdir = "inteiro_teor"
baixados = 0
for _, row in df_proposicoes.iterrows():
    if not row["url_pdf"]:
        continue
    time.sleep(0.15)  # cortesia
    if baixar_pdf(int(row["id"]), row["url_pdf"], outdir=outdir):
        baixados += 1

baixados, sorted([p.name for p in Path(outdir).glob("*.pdf")])[:10]


---
## 5) Lendo os PDFs e montando um **corpus**

Se `pdfminer.six` e `PyPDF2` estiverem instalados, extraímos texto e número de páginas. Caso contrário, o notebook segue sem quebrar.


In [ ]:
def ler_pdf(arquivo_pdf: str, pasta: str = "inteiro_teor"):
    caminho = Path(pasta) / arquivo_pdf
    # texto
    texto = ""
    if extract_text is not None:
        try:
            texto = extract_text(str(caminho)) or ""
        except Exception:
            texto = ""
    # páginas
    n_paginas = None
    if PdfReader is not None:
        try:
            reader = PdfReader(str(caminho))
            n_paginas = len(reader.pages)
        except Exception:
            n_paginas = None
    _id = arquivo_pdf.replace(".pdf", "")
    return {"id": _id, "n_paginas": n_paginas, "text": texto, "n_caracteres": len(texto)}

arquivos = sorted([p.name for p in Path("inteiro_teor").glob("*.pdf")])
corpus_docs = pd.DataFrame([ler_pdf(a) for a in arquivos])
corpus_docs.head()


---
## 6) **(Opcional)**: análises de texto (tokenização simples)


In [ ]:
stop_pt = set(stopwords.words('portuguese'))

def preprocess_text(s: str) -> str:
    s = (s or "").lower()
    s = unidecode(s)
    return s

tokens = []
for _, row in corpus_docs.iterrows():
    txt = preprocess_text(row.get("text", ""))
    words = [w for w in txt.split() if w.isalpha() and w not in stop_pt]
    tokens.extend(words)

top_palavras = (
    pd.Series(tokens, name="word")
    .value_counts()
    .head(30)
    .rename_axis("word")
    .reset_index(name="n")
)
top_palavras
